In [5]:
import os
os.chdir('..')
from inputGetter import get_input
from sortedcontainers import SortedList

content = get_input(2024, 6)[:-1]

Data preparation

In [6]:
content = [list(line) for line in content.split('\n')]
sz_r, sz_c = len(content), len(content[0])

# Basically, for each row/column, it will contain all columns/rows that has obstacble
obstacles_by_rows = [SortedList() for _ in range(sz_r)]
obstacles_by_columns = [SortedList() for _ in range(sz_c)]
start_r, start_c = -1, -1

for r in range(sz_r):
    for c in range(sz_c):
        if content[r][c] == '^':
            start_r, start_c = r, c
        if content[r][c] == '#':
            obstacles_by_rows[r].add(c)
            obstacles_by_columns[c].add(r)
            
directions = {'^': (-1, 0), 'v': (1, 0), '>': (0, 1), '<': (0, -1)}
turns = {'^': '>', '>': 'v', 'v': '<', '<': '^'}

Logic code for part B

In [ ]:
'''
Based on the current row, column, and direction info, this function would find the closest obstacle that it will reach. 
The finding will be O(log n)
There's add/sub 1 on row/col in depends on the direction because u would stop before touching the obstacle
'''
def get_new_position(direction, r, c):
    if direction == '^':
        row_id = obstacles_by_columns[c].bisect_left(r) - 1
        return (obstacles_by_columns[c][row_id] + 1 if row_id >= 0 else -1, c)
    
    elif direction == 'v':
        row_id = obstacles_by_columns[c].bisect_right(r)
        return (obstacles_by_columns[c][row_id] - 1 if row_id < len(obstacles_by_columns[c]) else sz_r, c)
    
    elif direction == '<':
        col_id = obstacles_by_rows[r].bisect_left(c) - 1
        return (r, obstacles_by_rows[r][col_id] + 1 if col_id >= 0 else -1)
    
    elif direction == '>':
        col_id = obstacles_by_rows[r].bisect_right(c)
        return (r, obstacles_by_rows[r][col_id] - 1 if col_id < len(obstacles_by_rows[r]) else sz_c)

'''Place an obstacle in a specific row and column, and check if it will cause a loop or not'''
def valid_obstacle(r, c):
    obstacles_by_rows[r].add(c)
    obstacles_by_columns[c].add(r)
    
    cpy_r, cpy_c = start_r, start_c
    visited = set()
    direction = '^'
    
    while True:
        # Break when either go out of bound (no loop) or go back to same place with same direction (loop exist)
        if cpy_r < 0 or cpy_r >= sz_r or cpy_c < 0 or cpy_c >= sz_c or (cpy_r, cpy_c, direction) in visited:
            break
        visited.add((cpy_r, cpy_c, direction))
        cpy_r, cpy_c = get_new_position(direction, cpy_r, cpy_c)
        direction = turns[direction]
    
    obstacles_by_rows[r].remove(c)
    obstacles_by_columns[c].remove(r)
    return 0 <= cpy_r < sz_r and 0 <= cpy_c < sz_c

Main logic

In [8]:
direction = '^'
cpy_r, cpy_c = start_r, start_c
res_a, res_b = 1, 0     # res_a start off with 1 because the starting point isn't counted in the loop

while True:
    next_r, next_c = cpy_r + directions[direction][0], cpy_c + directions[direction][1]
    if next_r < 0 or next_r >= sz_r or next_c < 0 or next_c >= sz_c:
        break
    if content[next_r][next_c] == '#':
        direction = turns[direction]
    else:
        cpy_r, cpy_c = next_r, next_c
        # Use the map itself to mark visited cell
        if content[cpy_r][cpy_c] == '.':
            res_b += valid_obstacle(cpy_r, cpy_c)
            res_a += 1
        content[cpy_r][cpy_c] = direction

print(f'Part A: {res_a}')
print(f'Part B: {res_b}')

Part A: 4663
Part B: 1530
